# Import packages

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install datasets

In [ ]:
import math
import pandas as pd
import datasets
import torch
from torch import nn
from tqdm import tqdm

# Dataset


In [ ]:
from datasets import load_dataset

dataset = load_dataset("cnn_dailymail", '3.0.0')

In [ ]:
dataset

In [ ]:
# dataset type 'datsaet'
print(type(dataset))

In [ ]:
# convert to pandas dataframe
df_cnn_train = pd.DataFrame(dataset['train'])
df_cnn_valid = pd.DataFrame(dataset['validation'])
df_cnn_test = pd.DataFrame(dataset['test'])

In [ ]:
# concatenate to make full dataframe
## 데이터셋 전체에 작업해서 보내드리면 되는 것 같아서, 일단 전부 concat해뒀습니다.
df_cnn = pd.concat([df_cnn_train, df_cnn_valid, df_cnn_test], axis=0)
df_cnn = df_cnn.reset_index(drop=True)
df_cnn.head()

In [ ]:
df_cnn.shape

# Model

#### 1. Model & GPU device Setting

In [ ]:
!pip install transformers

In [ ]:
# load tokenizer and model
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("bloomberg/KeyBART")

model = AutoModelForSeq2SeqLM.from_pretrained("bloomberg/KeyBART")

In [ ]:
model.eval()
model.to("cuda")

#### 2. Find better decoding methods

In [ ]:
# encode and check device
doc_input_ids = tokenizer.encode(df_cnn['article'][0], return_tensors="pt").cuda()
doc_input_ids.is_cuda

In [ ]:
# keyword case1. beam search 
doc_generated_sequence_1 = model.generate(
    input_ids=doc_input_ids, 
    max_length=50, 
    num_beams=5,
    early_stopping=True, #EOS토큰이 나오면 생성을 중단
    no_repeat_ngram_size=2 # 2-gram의 어구가 반복되지 않도록 설정함
    )

In [ ]:
# keyword case2. Top-k Sampling & Top-p (nucleus) sampling
doc_generated_sequence_2 = model.generate(
    input_ids=doc_input_ids,
    do_sample=True, #샘플링 전략 사용
    max_length=30, # 최대 디코딩 길이는 30
    top_k=60, # 확률 순위가 50위 밖인 토큰은 샘플링에서 제외
    top_p=0.95 # 누적 확률이 95%인 후보집합에서만 생성
    )

In [ ]:
# compare generated texts
doc_output_1 = tokenizer.decode(doc_generated_sequence_1.squeeze(), skip_special_tokens=True)
doc_output_2 = tokenizer.decode(doc_generated_sequence_2.squeeze(), skip_special_tokens=True)
ls_doc_output_1 = list(filter(None, doc_output_1.split(';')))
ls_doc_output_2 = list(filter(None, doc_output_2.split(';')))
print(f"""OUPUT OF THE FIRST CASE : {ls_doc_output_1} \n
OUPUT OF THE SECOND CASE : {ls_doc_output_2}""")
# trust fund;box office;gossip column;sports car;media scrutiny;world wide

**-> 두 번째 방식 채택**

In [ ]:
# try for more documents
sample_corpus = df_cnn.article[:5]

sample_keywords = []
for doc in sample_corpus:
    input_ids = tokenizer.encode(doc, padding=True, return_tensors="pt").cuda()
    if input_ids.is_cuda == True:
        generated_sequence = model.generate(
            input_ids=input_ids,
            do_sample=True, #샘플링 전략 사용
            max_length=30, # 최대 디코딩 길이는 30
            top_k=50, # 확률 순위가 50위 밖인 토큰은 샘플링에서 제외
            top_p=0.95 # 누적 확률이 95%인 후보집합에서만 생성
        )
        keywords = tokenizer.decode(generated_sequence.squeeze(), skip_special_tokens=True)
        ls_keywords = list(filter(None, keywords.split(';')))
        sample_keywords.append(ls_keywords)
    else :
        print('Error occured.')
        break

In [ ]:
 sample_keywords

#### 3. Extract keywords

In [ ]:
corpus = list(df_cnn['article'])

In [ ]:
all_keywords = []

sample_keywords = []
for doc in sample_corpus:
    input_ids = tokenizer.encode(doc, padding=True, return_tensors="pt").cuda()
    if input_ids.is_cuda == True:
        generated_sequence = model.generate(
            input_ids=input_ids,
            do_sample=True, #샘플링 전략 사용
            max_length=30, # 최대 디코딩 길이는 30
            top_k=50, # 확률 순위가 50위 밖인 토큰은 샘플링에서 제외
            top_p=0.95 # 누적 확률이 95%인 후보집합에서만 생성
        )
        keywords = tokenizer.decode(generated_sequence.squeeze(), skip_special_tokens=True)
        ls_keywords = list(filter(None, keywords.split(';')))
        sample_keywords.append(ls_keywords)
    else :
        print('Error occured.')
        break

### reference (bert) (temp)

In [ ]:
from transformers import AutoModel, AutoTokenizer

model_name = "distilroberta-base"
model_2 = AutoModel.from_pretrained(model_name)
tokenizer_2 = AutoTokenizer.from_pretrained(model_name)

In [ ]:
candidate_tokens_2 = tokenizer_2(candidates_list[0], padding=True, return_tensors="pt")
candidate_embeddings_2 = model_2(**candidate_tokens_2)["pooler_output"]

In [ ]:
candidate_tokens_2.input_ids.shape

In [ ]:
candidate_embeddings_2.shape

# Ouput

In [ ]:
# save as json format